In [1]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdQt8qhichgaF2Y9oaagTy0jwFq1iB1AxfUpJUoGs25SpwmDyle9cuhFau0



Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [4]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()

In [10]:
# get list of cities

#boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/geo_ref.csv')
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')

boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2
9,Musanze,RWA-Musanze,ADM5union,ADM5,RWA-Musanze-ADM5,RWA,ADM5


# Extract ESA raster by feature

In [11]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for Land cover as GeoTIFF
    geemap.ee_export_image_to_drive(
        esa_land_cover, 
        description = boundary_id + '-ESA-Landcover',
        folder='data', 
        scale=10, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

0
BRA-Salvador-ADM4union
1
COD-Bukavu-ADM3union
2
COD-Uvira-ADM3union
3
COG-Brazzaville-ADM4union
4
COL-Barranquilla-ADM4union
5
ETH-Addis_Ababa-ADM4union
6
ETH-Dire_Dawa-ADM3union
7
KEN-Nairobi-ADM3union
8
MEX-Mexico_City-ADM2union
9
RWA-Musanze-ADM5union
Could not convert the geojson to ee.Geometry()


Exception: Invalid GeoJSON geometry.

# Upload to aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [17]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-cities4forests' 

In [18]:
out_dir = os.getcwd()

In [19]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data\\' + boundary_id +'-ESA-Landcover.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/esa_world_cover/v_0/'+ boundary_id + '-ESA-world_cover-2020.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Salvador-ADM4union

 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: MEX-Mexico_City-ADM2union

 boundary_id: MEX-Monterrey-ADM2union


# Extract Built-up areas

In [28]:
# select built-up areas
esa_built_up = esa_land_cover.updateMask(esa_land_cover.eq(50))

In [29]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for Land cover as GeoTIFF
    geemap.ee_export_image_to_drive(
        esa_built_up, 
        description = boundary_id + '-ESA-Landcover_built-up',
        folder='data', 
        scale=10, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

0
BRA-Salvador-ADM4union
1
COD-Bukavu-ADM3union
2
COD-Uvira-ADM3union
3
COG-Brazzaville-ADM4union
4
COL-Barranquilla-ADM4union
5
ETH-Addis_Ababa-ADM4union
6
ETH-Dire_Dawa-ADM3union
7
KEN-Nairobi-ADM3union
8
MDG-Antananarivo-ADM4union
9
MEX-Mexico_City-ADM2union
10
MEX-Monterrey-ADM2union


# Upload in aws

In [30]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data\\' + boundary_id +'-ESA-Landcover_built-up.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/esa_world_cover/v_0/'+ boundary_id + '-ESA-world_cover-2020-built_up.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Salvador-ADM4union

 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: MEX-Mexico_City-ADM2union

 boundary_id: MEX-Monterrey-ADM2union
